# RAG Application with LlamaIndex

## Introduction

**LlamaIndex** is a data framework for your LLM applications. It is designed to easily connect custom data sources to your LLM. While LangChain is a general-purpose framework for building LLM applications (chains, agents, etc.), LlamaIndex optimizes for **indexing** and **retrieving** data.

### Key Differences form LangChain
- **Simpler Interface**: LlamaIndex often requires less code to get a RAG system up and running.
- **Data-Centric**: It focuses heavily on the "Index" part—structuring your data for optimal retrieval.

In this notebook, we will build a RAG application to chat with our PDF documents using LlamaIndex.

## Step 1: Install Dependencies

We need to install `llama-index`.

In [9]:
# Install llama-index
!uv pip install -q llama-index

## Step 2: Setup Environment

LlamaIndex uses OpenAI by default. We need to make sure our API key is set.

In [ ]:
# Option 1: Set your API key directly (for Colab)
#from google.colab import userdata
#os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Option 2: Load from .env file
# import os
# from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️ Warning: OPENAI_API_KEY not found. Please set it in your .env file.")
else:
    print("✅ OPENAI_API_KEY loaded")

✅ OPENAI_API_KEY loaded


## Step 3: Load Data

LlamaIndex makes data loading very simple with `SimpleDirectoryReader`. It automatically figures out how to parse files in a directory (PDFs, text files, etc.).

In [ ]:
# !pip install gdown -q

# !gdown 12hCcDOBYO0A3q2eFstMQnNqcCX2WYCZq -O pdfs.zip
# !unzip -q pdfs.zip -d pdfs
# !rm pdfs.zip
# print("✓ PDFs extracted to ./pdfs folder")

In [11]:
from llama_index.core import SimpleDirectoryReader

# Load data from the 'pdfs' directory
reader = SimpleDirectoryReader(input_dir="pdfs")
documents = reader.load_data()

print(f"✅ Loaded {len(documents)} document pages")

✅ Loaded 126 document pages


## Step 4: Create Index

Now we create a `VectorStoreIndex` from our documents. This effectively:
1. Chunks the documents.
2. Embeds the chunks using OpenAI embeddings.
3. Stores them in an in-memory vector store (by default).

> **Note**: Since this is in-memory, if you restart the kernel, the index is lost and needs to be recreated (incurring embedding costs again). We will see how to persist it later.

In [12]:
from llama_index.core import VectorStoreIndex

# Create the index (this happens in one line!)
index = VectorStoreIndex.from_documents(documents)

print("✅ Index created")

2025-12-09 13:57:24,078 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-09 13:57:27,500 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-09 13:57:29,657 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


✅ Index created


## Step 5: Querying

To ask questions, we create a "Query Engine" from our index. This engine handles the retrieval and LLM generation loop for us.

In [13]:
# Create a query engine
query_engine = index.as_query_engine()

# Ask a question
response = query_engine.query("What is Byte Pair Encoding?")

print("Response:")
print(response)

2025-12-09 13:57:33,048 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-09 13:57:35,020 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response:
Byte Pair Encoding (BPE) is a data compression technique that iteratively replaces the most frequent pair of symbols with a single, unused symbol. It is commonly used for subword tokenization in natural language processing tasks to create a token dictionary of a specified size.


### Inspecting the Source

We can check which documents were used to generate the answer.

In [14]:
for node in response.source_nodes:
    print("--------------------------------------------------")
    print(f"Score: {node.score}")
    print(f"Source File: {node.metadata.get('file_name')}")
    print(f"Content: {node.text[:200]}...")

--------------------------------------------------
Score: 0.8890902737390015
Source File: 2411.08671v1.pdf
Content: arXiv:2411.08671v1  [cs.DS]  13 Nov 2024
Theoretical Analysis of Byte-Pair Encoding
L´ aszl´ o Kozma and Johannes Voderholzer
Institut f¨ ur Informatik, Freie Universit¨ at Berlin, Germany
Abstract
By...
--------------------------------------------------
Score: 0.8741710360039733
Source File: 1508.07909v5.pdf
Content: As an alternative, we pro-
pose a segmentation algorithm based on byte pair
encoding (BPE), which lets us learn a vocabulary
that provides a good compression rate of the text.
3.2 Byte Pair Encoding (...


## Conclusion

As you can see, LlamaIndex abstracts away a lot of the complexity (chunking, embedding, retrieval setup) that we explicitly handled in the LangChain version. This makes it a great choice for getting up and running quickly with standard RAG pipelines.

## Persisting to Disk

By default, LlamaIndex stores the data in-memory. This means if you restart the kernel, you lose the index and have to rebuild it (re-embedding everything, which costs money).

We can persist the index to disk to avoid this.

In [15]:
# Persist the index to a directory
index.storage_context.persist(persist_dir="./vector_db_llama")

print("✅ Index persisted to ./vector_db_llama")

✅ Index persisted to ./vector_db_llama


### Loading from Disk

To load the index back, we use `StorageContext` and `load_index_from_storage`.

In [16]:
from llama_index.core import StorageContext, load_index_from_storage

# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./vector_db_llama")

# Load index from the storage context
loaded_index = load_index_from_storage(storage_context)
print("✅ Index loaded from disk")

# Verify it works
query_engine_loaded = loaded_index.as_query_engine()
response = query_engine_loaded.query("What is Byte Pair Encoding?")
print(response)

2025-12-09 13:57:44,245 - INFO - Loading all indices.


✅ Index loaded from disk


2025-12-09 13:57:45,367 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-09 13:57:46,339 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Byte Pair Encoding (BPE) is a data compression technique that involves iteratively replacing the most frequent pair of symbols with a new symbol. This process helps in shortening the text by creating a lookup-table of new symbols and the pairs they replace, allowing for the reconstruction of the original text. The goal of BPE is to achieve the best compression within a predefined budget by fixing the number of new symbols upfront.


## 🎓 Student Challenge

Now it's your turn! 

**Goal**: Create a new RAG system using your *own* PDF documents.

1. Create a new folder named `student_pdfs` in this directory.
2. Upload 1-2 PDF files into that folder (e.g., a paper, a resume, a manual).
3. Use `SimpleDirectoryReader` to load documents from `student_pdfs`.
4. Create a new `VectorStoreIndex` from these documents.
5. Create a query engine and ask a question about your documents.

Write your code in the cells below.

In [ ]:
# 1. Load your documents from 'student_pdfs'


In [ ]:
# 2. Create an index


In [ ]:
# 3. Query your index
